In [14]:
import random
from instances import get_instances
import numpy as np
import pandas as pd
from heuristique_de_reparation import reparation, gen_sol_initiale
from structure_de_voisinage import gen_voisin_perm
from meta_genetique import genetic_algo
import json
import tqdm

In [15]:
files = [f'instances/mknapcb{i}' for i in range(1, 10)]

In [ ]:
df = pd.DataFrame(columns=['file','instance', 'opt_value', 'value_genetic', 'gap'])
with tqdm(total=len(files), desc="Traitement des fichiers", unit="fichier") as overall_progress:
    for file in files : 
        inst_file = file + '.txt'
        with open(file + '_sol.json', 'r') as f:
            opt_val_file = list(json.load(f).values())
        
        instances = get_instances(inst_file)
        with tqdm(total=len(instances), desc=f"Traitement de {file}", leave=True, unit="instance") as file_progress:
            for i in range(len(instances)):
                ins = instances[i]
                ins["opt_value"] = opt_val_file[i]
                
                c = ins["gains"]
                a = np.array(ins["ressources"])
                b = ins["quantite_ressources"]
                N = len(c)
                M = len(b)
                x_best, best_value = genetic_algo(N, M, c, a, b, max_iter=200, pop_size=100)
                infos = infos = {
                    'file': [file.split('/')[-1]],
                    'instance': [i],
                    'opt_value': [float(ins["opt_value"])],
                    'value_genetic': [best_value],
                    'gap':[ (float(ins["opt_value"]) - best_value) / float(ins["opt_value"])*100]
                }
                df = pd.concat([df, pd.DataFrame(infos)])
                file_progress.update(1)
        overall_progress.update(1)

    

File : instances/mknapcb1, Instance : 0


C:\Users\AHMED-YASSINE\AppData\Local\Temp\ipykernel_7460\586467121.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(infos)])


File : instances/mknapcb1, Instance : 1
File : instances/mknapcb1, Instance : 2
File : instances/mknapcb1, Instance : 3
File : instances/mknapcb1, Instance : 4
File : instances/mknapcb1, Instance : 5
File : instances/mknapcb1, Instance : 6
File : instances/mknapcb1, Instance : 7
File : instances/mknapcb1, Instance : 8
File : instances/mknapcb1, Instance : 9
File : instances/mknapcb1, Instance : 10
File : instances/mknapcb1, Instance : 11
File : instances/mknapcb1, Instance : 12
File : instances/mknapcb1, Instance : 13
File : instances/mknapcb1, Instance : 14
File : instances/mknapcb1, Instance : 15
File : instances/mknapcb1, Instance : 16
File : instances/mknapcb1, Instance : 17
File : instances/mknapcb1, Instance : 18
File : instances/mknapcb1, Instance : 19
File : instances/mknapcb1, Instance : 20
File : instances/mknapcb1, Instance : 21
File : instances/mknapcb1, Instance : 22
File : instances/mknapcb1, Instance : 23
File : instances/mknapcb1, Instance : 24
File : instances/mknapcb1